<a href="https://colab.research.google.com/github/rhnfzl/2IMM10-DeepLearnig-Assignments/blob/master/Assignment%20III/Assignment-3.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 4. Sequence Classification

<b>Group [30]</b>
* <b> Student </b> : Rehan Fazal + 1423002

# Task: Aspect-level Sentiment Classification(10pt)

Reading material:
- [1] R. He, WS. Lee & D. Dahlmeier. Exploiting document knowledge for aspect-level sentiment classification. 2018. https://arxiv.org/abs/1806.04346.


Build an attention-based aspect-level sentiment classification model with biLSTM. Your model shall include:

- BiLSTM network that learns sentence representation from input sequences.
- Attention network that assigns attention score over a sequence of biLSTM hidden states based on aspect terms representation.
- Fully connected network that predicts sentiment label, given the representation weighted by the attention score.

Requirements:

- You shall train your model bsaed on transferring learning. That is, you need first train your model on documnet-level examples. Then the learned weights will be used to initialize aspect-level model and fine tune it on aspect-level examples.
- You shall use the alignment score function in attention network as following expression:$$f_{score}(h,t)=tanh(h^TW_a t)$$
- You shall evaluate the trained model on the provided test set and show the accuracy on test set.


In [0]:
from google.colab import drive
drive.mount('/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /drive


In [0]:
import os
import sys
import codecs
import operator
import numpy as np
import re
from time import time

In [0]:
import _pickle as cPickle


#Load Data

In [0]:
def read_pickle(data_path, file_name):

    f = open(os.path.join(data_path, file_name), 'rb')
    read_file = cPickle.load(f)
    f.close()

    return read_file

def save_pickle(data_path, file_name, data):

    f = open(os.path.join(data_path, file_name), 'wb')
    cPickle.dump(data, f)
    print(" file saved to: %s"%(os.path.join(data_path, file_name)))
    f.close()

In [0]:
aspect_path = '/drive/My Drive/Deep Learning Course/practice-5-data/aspect-level/'


vocab = read_pickle(aspect_path, 'all_vocab.pkl')

train_x = read_pickle(aspect_path, 'train_x.pkl')
train_y = read_pickle(aspect_path, 'train_y.pkl')
dev_x = read_pickle(aspect_path, 'dev_x.pkl')
dev_y = read_pickle(aspect_path, 'dev_y.pkl')
test_x = read_pickle(aspect_path, 'test_x.pkl')
test_y = read_pickle(aspect_path, 'test_y.pkl')

train_aspect = read_pickle(aspect_path, 'train_aspect.pkl')
dev_aspect = read_pickle(aspect_path, 'dev_aspect.pkl')
test_aspect = read_pickle(aspect_path, 'test_aspect.pkl')


pretrain_data = read_pickle(aspect_path, 'pretrain_data.pkl')
pretrain_label = read_pickle(aspect_path, 'pretrain_label.pkl')

In [0]:
class Dataiterator_doc():
    '''
      1) Iteration over minibatches using next(); call reset() between epochs to randomly shuffle the data
      2) Access to the entire dataset using all()
    '''
    
    def __init__(self, X, y, seq_length=32, decoder_dim=300, batch_size=32):      
        self.X = X 
        self.y = y 
        self.num_data = len(X) # total number of examples
        self.batch_size = batch_size # batch size
        self.reset() # initial: shuffling examples and set index to 0
    
    def __iter__(self): # iterates data
        return self


    def reset(self): # initials
        self.idx = 0
        self.order = np.random.permutation(self.num_data) # shuffling examples by providing randomized ids 
        
    def __next__(self): # return model inputs - outputs per batch
        X_ids = [] # hold ids per batch 
        while len(X_ids) < self.batch_size:
            X_id = self.order[self.idx] # copy random id from initial shuffling
            X_ids.append(X_id)
            self.idx += 1 # 
            if self.idx >= self.num_data: # exception if all examples of data have been seen (iterated)
                self.reset()
                raise StopIteration()
                
        batch_X = self.X[np.array(X_ids)] # X values (encoder input) per batch
        batch_y = self.y[np.array(X_ids)] # y_in values (decoder input) per batch
        return batch_X, batch_y

          
    def all(self): # return all data examples
        return self.X, self.y
class Dataiterator_aspect():
    '''
      1) Iteration over minibatches using next(); call reset() between epochs to randomly shuffle the data
      2) Access to the entire dataset using all()
    '''
    
    def __init__(self, aspect_data, seq_length=32, decoder_dim=300, batch_size=32):
        
        len_aspect_data = len(aspect_data[0])
        #self.len_doc_data = len(doc_data[0])
        
        self.X_aspect = aspect_data[0] 
        self.y_aspect = aspect_data[1]
        self.aspect_terms = aspect_data[2]  
        self.num_data = len_aspect_data
        self.batch_size = batch_size # batch size
        self.reset() # initial: shuffling examples and set index to 0
    
    def __iter__(self): # iterates data
        return self


    def reset(self): # initials
        self.idx = 0
        self.order = np.random.permutation(self.num_data) # shuffling examples by providing randomized ids 
        
    def __next__(self): # return model inputs - outputs per batch
        
        X_ids = [] # hold ids per batch 
        while len(X_ids) < self.batch_size:
            X_id = self.order[self.idx] # copy random id from initial shuffling
            X_ids.append(X_id)
            self.idx += 1 # 
            if self.idx >= self.num_data: # exception if all examples of data have been seen (iterated)
                self.reset()
                raise StopIteration()
                
        batch_X_aspect = self.X_aspect[np.array(X_ids)] # X values (encoder input) per batch
        batch_y_aspect = self.y_aspect[np.array(X_ids)] # y_in values (decoder input) per batch
        batch_aspect_terms = self.aspect_terms[np.array(X_ids)]
        
        return batch_X_aspect, batch_y_aspect, batch_aspect_terms

          
    def all(self): # return all data examples
        return self.X_aspect, self.y_aspect, self.aspect_terms

In [0]:
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Lambda, Dropout, LSTM,Bidirectional
from keras.layers import Reshape, Activation, RepeatVector, concatenate, Concatenate, Dot, Multiply
import keras.backend as K
from keras.engine.topology import Layer
from keras import initializers
from keras import regularizers
from keras import constraints

Using TensorFlow backend.


In [0]:
overal_maxlen = 82
overal_maxlen_aspect = 7


#Define Attention Network Layer
- Define class for Attention Layer
- You need to finish the code for calculating the attention weights

In [0]:
class Attention(Layer):
    def __init__(self,  **kwargs):
        """
        Keras Layer that implements an Content Attention mechanism.
        Supports Masking.
        """
       
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert type(input_shape) == list
       
        self.steps = input_shape[0][1]

        self.W = self.add_weight(shape=(input_shape[0][-1], input_shape[1][-1]),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),)

        self.built = True

    def compute_mask(self, input_tensor, mask=None):
        assert type(input_tensor) == list
        assert type(mask) == list
        return None

    def call(self, input_tensor, mask=None):
        x = input_tensor[0] #Output of BiLSTM of sentence
        aspect = input_tensor[1] #Output of word embedding layer of aspect terms
        mask = mask[0] #Used to remove the influence of padded value
        ###YOUR CODE HERE###
        #From the [1] git repo https://github.com/ruidan/Aspect-level-sentiment/blob/master/code/my_layers.py#L40
        aspect = K.transpose(K.dot(self.W, K.transpose(aspect)))
        aspect = K.expand_dims(aspect, axis=-2)
        aspect = K.repeat_elements(aspect, self.steps, axis=1)
        eij = K.sum(x*aspect, axis=-1)

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        return a

   
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1])

In [0]:
class Average(Layer):
  
    def __init__(self, mask_zero=True, **kwargs):
        self.mask_zero = mask_zero
        self.supports_masking = True
        super(Average, self).__init__(**kwargs)

    def call(self, x,mask=None):
        if self.mask_zero:           
            mask = K.cast(mask, K.floatx())
            mask = K.expand_dims(mask)
            x = x * mask
            return K.sum(x, axis=1) / (K.sum(mask, axis=1) + K.epsilon())
        else:
            return K.mean(x, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])
    
    def compute_mask(self, x, mask):
        return None

#Establish computation Grah for model
- Input tensors
- Shared WordEmbedding layer 
- Attention network layer  
- Shared BiLSTM layer
- Shared fully connected layer(prediction layer)


In [0]:
dropout = 0.5     
recurrent_dropout = 0.1
vocab_size = len(vocab)
num_outputs = 3 # labels

##Input tensors for Aspect level model and document level model

In [0]:
#YOUR CODE HERE ##### Inputs #####
sentence_input = Input(shape=(overal_maxlen,), dtype='int32', name='sentence_input')  #input for sentence from aspect-level data
aspect_input = Input(shape=(overal_maxlen_aspect,), dtype='int32', name='aspect_input') #input for aspect terms
pretrain_input = Input(shape=(None,), dtype='int32', name='pretrain_input')             #input for sentence from document-level data
print('sentence_input : ',  sentence_input)
print('aspect_input   : ', aspect_input)
print('pretrain_input : ', pretrain_input)  

sentence_input :  Tensor("sentence_input:0", shape=(None, 82), dtype=int32)
aspect_input   :  Tensor("aspect_input:0", shape=(None, 7), dtype=int32)
pretrain_input :  Tensor("pretrain_input:0", shape=(None, None), dtype=int32)


##Shared WordEmbedding layer (Used for both Aspect and Document Level Embedding)

In [0]:
#YOUR CODE HERE
##### create word embedding layer #####
word_emb = Embedding(vocab_size, 300, mask_zero=True, name='word_emb')
### represent aspect as averaged word embedding ###
aspect_term_embs = word_emb(aspect_input)
aspect_embs = Average(mask_zero=True, name='aspect_emb')(aspect_term_embs)  #There could be mutiple words for aspect terms, here is to average the wordembedding of these aspect terms
### sentence representation ###
sentence_embs = word_emb(sentence_input) # from aspect-level domain
pretrain_embs = word_emb(pretrain_input) # from document-level domain

print('aspect_term_embs : ', aspect_term_embs)
print('aspect_embs      : ', aspect_embs)
print('sentence_embs    : ', sentence_embs)
print('pretrain_embs    : ', pretrain_embs)

aspect_term_embs :  Tensor("word_emb/embedding_lookup/Identity_1:0", shape=(None, 7, 300), dtype=float32)
aspect_embs      :  Tensor("aspect_emb/truediv:0", shape=(None, 300), dtype=float32)
sentence_embs    :  Tensor("word_emb_1/embedding_lookup/Identity_1:0", shape=(None, 82, 300), dtype=float32)
pretrain_embs    :  Tensor("word_emb_2/embedding_lookup/Identity_1:0", shape=(None, None, 300), dtype=float32)


##Shared BiLSTM layer

In [0]:
#YOUR CODE HERE 
rnn = Bidirectional(LSTM(300, return_sequences=True, dropout=dropout, recurrent_dropout=recurrent_dropout), name='BiLSTM')
### sentence representation ###
pretrain_bilstm = rnn(pretrain_embs)    # from document-level domain
sentence_bilstm = rnn(sentence_embs)    # from aspect-level domain
print('Shape of pretrain_bilstm : ', pretrain_bilstm.shape)
print('Shape of sentence_bilstm : ', sentence_bilstm.shape)

Shape of pretrain_bilstm :  (None, None, 600)
Shape of sentence_bilstm :  (None, 82, 600)


##Attention Layer

In [0]:
##YOUR CODE HERE
att_weights = Attention()([sentence_bilstm, aspect_embs])
print('att_weights : ', att_weights)

att_weights :  Tensor("attention_1/truediv:0", shape=(None, 82), dtype=float32)


##Shared Prediction Layer

In [0]:
#YOUR CODE HERE
dotproduct = Dot(axes=1, name = "dotproduct")([sentence_bilstm, att_weights])
pretrain_avg = Average(mask_zero=True)(pretrain_bilstm)
sentence_output = Dense(num_outputs, activation='softmax', name='dense_1')(dotproduct)  #for aspect-level
pretrain_output = Dense(num_outputs, activation='softmax', name='dense_2')(pretrain_avg)  # for document-level
#sentence_output = Reshape((num_outputs,))(sentence_output)          # to squeeze the shape from (None,1,3) to (None,3)

#Build Models for document-level and aspect-level data
- The two models shared the embedding, BiLSTM, Prediction Layer

In [0]:
### YOUR CODE HERE
model1 = Model(inputs=[pretrain_input], outputs=[pretrain_output]) #document level model
model2 = Model(inputs=[sentence_input, aspect_input], outputs=[sentence_output]) #aspect level model

In [0]:
model1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
pretrain_input (InputLayer)  (None, None)              0         
_________________________________________________________________
word_emb (Embedding)         multiple                  3000900   
_________________________________________________________________
BiLSTM (Bidirectional)       multiple                  1442400   
_________________________________________________________________
average_1 (Average)          (None, 600)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 1803      
Total params: 4,445,103
Trainable params: 4,445,103
Non-trainable params: 0
_________________________________________________________________


In [0]:
model2.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sentence_input (InputLayer)     (None, 82)           0                                            
__________________________________________________________________________________________________
aspect_input (InputLayer)       (None, 7)            0                                            
__________________________________________________________________________________________________
word_emb (Embedding)            multiple             3000900     aspect_input[0][0]               
                                                                 sentence_input[0][0]             
__________________________________________________________________________________________________
BiLSTM (Bidirectional)          multiple             1442400     word_emb[1][0]             

#Train Model
- First Train model on document-level data.
- Then Train  model on aspect-level data

##Train on document-level data (model1)

In [0]:
import keras.optimizers as opt
optimizer=opt.RMSprop(lr=0.001, rho=0.9, epsilon=1e-06, clipnorm=10, clipvalue=0)
model1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
batch_size = 128
train_steps_epoch = len(pretrain_data)/batch_size
batch_train_iter_doc = Dataiterator_doc(pretrain_data, pretrain_label, batch_size)

In [0]:
###YOUR CODE HERE###
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train_generator(model, batch_train_iter):
    
    def train_gen():
        while True:
            train_batches = [[X, y] for X, y in batch_train_iter]
            for train_batch in train_batches:
                yield train_batch
                
    history1 = model.fit_generator(train_gen(), \
                                  steps_per_epoch=train_steps_epoch, \
                                  epochs = 10)

In [0]:
train_generator(model1, batch_train_iter_doc)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
235/234 [==============================] - 558s 2s/step - loss: 0.9421 - categorical_accuracy: 0.5484
Epoch 2/10
235/234 [==============================] - 545s 2s/step - loss: 0.7962 - categorical_accuracy: 0.6376
Epoch 3/10
235/234 [==============================] - 538s 2s/step - loss: 0.7558 - categorical_accuracy: 0.6594
Epoch 4/10
235/234 [==============================] - 538s 2s/step - loss: 0.7170 - categorical_accuracy: 0.6798
Epoch 5/10
235/234 [==============================] - 538s 2s/step - loss: 0.6508 - categorical_accuracy: 0.7145
Epoch 6/10
235/234 [==============================] - 538s 2s/step - loss: 0.6725 - categorical_accuracy: 0.7093
Epoch 7/10
235/234 [==============================] - 536s 2s/step - loss: 0.6558 - categorical_accuracy: 0.7146
Epoch 8/10
235/234 [==============================] - 534s 2s/step - loss: 0.6717 - categorical_accuracy: 0.7144
Epoch 9/10
235/234 [==============================] - 537s 2s/step - loss: 0.5987 - categorical_

##Train on aspect-level data (model2)

In [0]:
train_steps_epoch = len(train_x)/batch_size
batch_train_iter_aspect = Dataiterator_aspect([train_x, train_y, train_aspect], batch_size)
val_steps_epoch = len(dev_x)/batch_size
batch_val_iter_aspect = Dataiterator_aspect([dev_x, dev_y, dev_aspect], batch_size)

import keras.optimizers as opt
optimizer = opt.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, clipnorm=10, clipvalue=0)
model2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [0]:
### YOUR CODE HERE
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train_generator(model, batch_train_iter, batch_val_iter):
    
    earlystop_callbacks = [EarlyStopping(monitor='val_loss', patience=10),
                     ModelCheckpoint(filepath=os.path.join('./','{epoch:02d}-{loss:.2f}.check'), \
                                     monitor='val_loss', save_best_only=False, \
                                     save_weights_only=True)
                     ]
    
    def train_gen():
        while True:
            train_batches = [[[X, aspect], [y]] for X, y, \
                             aspect in batch_train_iter]
            for train_batch in train_batches:
                yield train_batch
                
    def val_gen():
        while True:
            val_batches = [[[X, aspect], [y]] for X, y, \
                           aspect in batch_val_iter]
            for val_batch in val_batches:
                yield val_batch
                
    history2 = model.fit_generator(train_gen(), validation_data=val_gen(), \
                                  validation_steps=val_steps_epoch, steps_per_epoch=train_steps_epoch, \
                                  epochs = 100, callbacks = earlystop_callbacks)

In [0]:
train_generator(model2, batch_train_iter_aspect, batch_val_iter_aspect)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
15/14 [===============================] - 11s 759ms/step - loss: 1.0993 - categorical_accuracy: 0.3458 - val_loss: 1.0393 - val_categorical_accuracy: 0.5078
Epoch 2/100
15/14 [===============================] - 10s 695ms/step - loss: 1.0096 - categorical_accuracy: 0.5562 - val_loss: 0.9055 - val_categorical_accuracy: 0.6797
Epoch 3/100
15/14 [===============================] - 10s 671ms/step - loss: 0.9283 - categorical_accuracy: 0.6313 - val_loss: 0.9098 - val_categorical_accuracy: 0.6328
Epoch 4/100
15/14 [===============================] - 10s 666ms/step - loss: 0.9182 - categorical_accuracy: 0.5938 - val_loss: 0.7933 - val_categorical_accuracy: 0.6641
Epoch 5/100
15/14 [===============================] - 10s 668ms/step - loss: 0.8424 - categorical_accuracy: 0.6396 - val_loss: 0.7116 - val_categorical_accuracy: 0.6172
Epoch 6/100
15/14 [===============================] - 10s 676ms/step - loss: 0.8610 - categorical_accuracy: 0.6042 - val_loss: 0.8073 - val_categorical_acc

##Evaluating on test set on model2
- show the accuracy

In [0]:
##YOUR CODE HERE
loss, accuracy = model2.evaluate([test_x, test_aspect], test_y)

638/638 [==============================] - 2s 3ms/step


In [0]:
print("Loss : ", loss)
print("Accuracy : ", (accuracy*100), "%")

Loss :  0.8247347302197663
Accuracy :  60.501569509506226 %
